In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

input_shape = x_train.shape[1:]

In [3]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', 
                       kernel_initializer='lecun_normal', 
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss='mse', optimizer='nadam', metrics=[create_huber(2.0)])
model.fit(x_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 497us/step - loss: 1.7421 - huber_fn: 0.8001
Epoch 2/2
363/363 [==============================] - 0s 500us/step - loss: 0.5119 - huber_fn: 0.2502


In [4]:
model.compile(loss=create_huber(2.0), optimizer='nadam', metrics=[create_huber(2.0)])

sample_weight = np.random.rand(len(y_train))
history = model.fit(x_train_scaled, y_train, epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 1s 566us/step - loss: 0.1165 - huber_fn: 0.2325
Epoch 2/2
363/363 [==============================] - 0s 583us/step - loss: 0.1121 - huber_fn: 0.2248


In [5]:
history.history['loss'][0], history.history['huber_fn'][0] * sample_weight.mean()

(0.11652007699012756, 0.11571321765544094)

Streaming metrics

In [6]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [7]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [8]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [9]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [10]:
precision.reset_states()

In [11]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight('total', initializer='zeros')
        self.count = self.add_weight('count', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

In [12]:
m = HuberMetric(2.)

m(tf.constant([[2.]]), tf.constant([[10.]]))

<tf.Tensor: shape=(), dtype=float32, numpy=14.0>

In [13]:
m(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [14]:
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=21.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>]

In [15]:
m.reset_states()
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=0.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=0.0>]

In [16]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', 
                       kernel_initializer='lecun_normal', 
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=create_huber(2.0), optimizer='nadam', metrics=[HuberMetric(2.0)])

model.fit(x_train_scaled.astype(np.float32), y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 533us/step - loss: 0.7368 - huber_metric_1: 0.7368
Epoch 2/2
363/363 [==============================] - 0s 525us/step - loss: 0.2379 - huber_metric_1: 0.2379


In [17]:
model.save('model_with_custom_metric.h5')

In [18]:
model = keras.models.load_model('model_with_custom_metric.h5', 
                                custom_objects={
                                    'huber_fn': create_huber(2.0),
                                    'HuberMetric': HuberMetric,
                                })

In [19]:
model.fit(x_train_scaled.astype(np.float32), y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 0s 514us/step - loss: 0.2271 - huber_metric_1: 0.2271
Epoch 2/2
363/363 [==============================] - 0s 517us/step - loss: 0.2217 - huber_metric_1: 0.2217


In [20]:
model.metrics[-1].threshold

2.0

In [21]:
class HuberMetric(keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)
        
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

In [22]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', 
                       kernel_initializer='lecun_normal', 
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=keras.losses.Huber(2.0), optimizer='nadam', metrics=[HuberMetric(2.0)])

sample_weight = np.random.rand(len(y_train))

history = model.fit(x_train_scaled.astype(np.float32), 
                    y_train.astype(np.float32), 
                    epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 1s 569us/step - loss: 0.4283 - HuberMetric: 0.8607
Epoch 2/2
363/363 [==============================] - 0s 583us/step - loss: 0.1207 - HuberMetric: 0.2409


In [23]:
history.history['loss'][0], history.history['HuberMetric'][0] * sample_weight.mean()

(0.4283002018928528, 0.4294615504130946)

In [24]:
model.save('model_with_custom_metric_v2.h5')

In [25]:
model = keras.models.load_model('model_with_custom_metric_v2.h5', 
                                custom_objects={'HuberMetric': HuberMetric})

In [26]:
model.fit(x_train_scaled.astype(np.float32), 
          y_train.astype(np.float32), 
          epochs=2)

Epoch 1/2
363/363 [==============================] - 0s 533us/step - loss: 0.2318 - HuberMetric: 0.2318
Epoch 2/2
363/363 [==============================] - 0s 539us/step - loss: 0.2245 - HuberMetric: 0.2245


In [27]:
model.metrics[-1].threshold

2.0